# Why this notebook?

After reading this post about [Pydicom vs DicomSDL (Fast dicom export and processing (1.6-2x faster) 💪)](https://www.kaggle.com/competitions/rsna-breast-cancer-detection/discussion/371033), I checked whether DicomSDL had the option to apply a VOI lookup table or windowing operation like Pydicom.<br>
It turns out, they don't. They only have this function in their source code 😅

    def apply_window_center():
        print("Hello")
        

## Apply_voi_lut

Next, I dug into the source code of Pydicom to see what they do exactly inside the apply_voi_lut method.<br>
The code checks several things, of which many don't apply for the images/datasets in the dicom images in this competition.<br>
For example, **none** of the datasets in this competition have a valid **VOILUTSequence**.<br>
This means the **apply_voi_lut function** will always redirect to the **apply_windowing**.<br>
Therefore, the next step was to check this function inside Pydicom's code.

## Apply_windowing

Here, the most important thing the code checks is which VOILUTFunction is used in the dataset.<br>
In this competition, 22% of the datasets (12139 of 54706) have the Sigmoid function, the rest uses the LINEAR function.<br>
As the code is very different for each of these functions, I recreated/copied the necessary code for both functions.<br>
Please note that I removed some code for speed, as many checks in the pydicom code is not not applicable in this competition.
For example:
- Pixelrepresentation and ModalityLUTSequence is always NaN
- Window width/center is always set correctly
- RescaleSlope & RescaleIntercept is always the same


## Results/conclusion

As you can see in the results below, using the VOILUTFunction is quite important.<br>
Around 22% of the images uses SIGMOID instead of LINEAR, and the images/colors for Sigmoid are very different, a lot lighter/more yellow.<br>
But once you apply the windowing function, they seem completely similar to images with the Linear VOILUTFunction.<br>
**Therefore**, if you use DicomSDL instead of Pydicom, you should definitely create your own apply_windowing function like in Pydicom!


### ⬇ Libraries

In [ ]:
# Installs
!pip install -qU "python-gdcm" pydicom pylibjpeg "opencv-python-headless"
!pip install /kaggle/input/dicomsdl/dicomsdl-0.109.1-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl

# Imports
import os
import time
import pydicom
import dicomsdl as pysdl

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
from pydicom.pixel_data_handlers.util import apply_voi_lut

### Loading images with Pydicom & DicomSDL

In [ ]:
def load_image_pydicom(img_path, voi_lut=False):
    dataset = pydicom.dcmread(img_path)
    img = dataset.pixel_array
    if voi_lut:
        img = apply_voi_lut(img, dataset)
    if dataset.PhotometricInterpretation == "MONOCHROME1":
        img = np.amax(img) - img
    return img


def load_image_dicomsdl(img_path, voi_lut=False):
    dataset = pysdl.open(img_path)
    img = dataset.pixelData()
    
    if voi_lut:
        # Load only the variables we need
        center = dataset["WindowCenter"]
        width = dataset["WindowWidth"]
        bits_stored = dataset["BitsStored"]
        voi_lut_function = dataset["VOILUTFunction"]

        # For sigmoid it's a list, otherwise a single value
        if isinstance(center, list):
            center = center[0]
        if isinstance(width, list):
            width = width[0]

        # Set y_min, max & range
        y_min = 0
        y_max = float(2**bits_stored - 1)
        y_range = y_max

        # Function with default LINEAR (so for Nan, it will use linear)
        if voi_lut_function == "SIGMOID":
            img = y_range / (1 + np.exp(-4 * (img - center) / width)) + y_min
        else:
            # Checks width for < 1 (in our case not necessary, always >= 750)
            center -= 0.5
            width -= 1

            below = img <= (center - width / 2)
            above = img > (center + width / 2)
            between = np.logical_and(~below, ~above)

            img[below] = y_min
            img[above] = y_max
            if between.any():
                img[between] = (
                    ((img[between] - center) / width + 0.5) * y_range + y_min
                )
    
    if dataset["PhotometricInterpretation"] == "MONOCHROME1":
        img = np.amax(img) - img

    return img

# Resulting images with VOILUTFunction -> Linear

As you can see below, images with VOILUTFunction Linear doesn't change at all.

In [ ]:
img_path_linear = "../input/rsna-breast-cancer-detection/train_images/10102/1181635673.dcm"
fig, axs = plt.subplots(nrows=1, ncols=4,figsize=(30, 16))

img = load_image_pydicom(img_path_linear)
plt.subplot(141)
plt.title("Pydicom - Linear without VOI-LUT")
plt.imshow(img)

img = load_image_dicomsdl(img_path_linear)
plt.subplot(142)
plt.title("DicomSDL - Linear without VOI-LUT")
plt.imshow(img)

img = load_image_pydicom(img_path_linear, True)
plt.subplot(143)
plt.title("Pydicom - Linear with VOI-LUT")
plt.imshow(img)

img = load_image_dicomsdl(img_path_linear, True)
plt.subplot(144)
plt.title("DicomSDL - Linear with VOI-LUT")
plt.imshow(img)

plt.show()

# Resulting images with VOILUTFunction -> Sigmoid

For VOILUTFunction Sigmoid, however, it does have an effect. You can clearly see that the default images for Sigmoid without VOILUT are a lot more light/yellow, and once VOILUT is applied the colors seem more similar to images with VOILUTFunction Linear.

In [ ]:
img_path_sigmoid = "../input/rsna-breast-cancer-detection/train_images/10006/1459541791.dcm"
fig, axs = plt.subplots(nrows=1, ncols=4,figsize=(30, 16))

img = load_image_pydicom(img_path_sigmoid)
plt.subplot(141)
plt.title("Pydicom - Sigmoid without VOI-LUT")
plt.imshow(img)

img = load_image_dicomsdl(img_path_sigmoid)
plt.subplot(142)
plt.title("DicomSDL - Sigmoid without VOI-LUT")
plt.imshow(img)

img = load_image_pydicom(img_path_sigmoid, True)
plt.subplot(143)
plt.title("Pydicom - Sigmoid with VOI-LUT")
plt.imshow(img)

img = load_image_dicomsdl(img_path_sigmoid, True)
plt.subplot(144)
plt.title("DicomSDL - Sigmoid with VOI-LUT")
plt.imshow(img)

plt.show()